In [4]:
from core.utils import Tibanna
from core import ff_utils

env = 'fourfront-webdev'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

publications = [
                ["cf0e49aa-173c-49d1-a7c7-22acbc83c064","Rao_et_al"],
                ["5427c34d-1ff4-409c-9ce1-97c7a5af2965","Sanborn_et_al"],
                ["088313f3-b588-4b71-8372-e202ba3baa97","Selvaraj_et_al"],
                ["da387ade-8822-4009-9269-90e2302c09d2","Jin_et_al"]
            ]

# Variables
pub_name = ''
set_id = ''
set_title = ''
cell_line = ''
mod = ''
treat = ''
bio = ''
tec = ''
enzyme = ''
file_alias = ''


for pub, pub_name in publications:
    exp_sets = ff_utils.get_metadata(pub, connection=ff).get('exp_sets_prod_in_pub')
    for exp_set in exp_sets:
        set_info = ff_utils.get_metadata(exp_set, connection=ff)
        # skip custom sets
        if set_info['experimentset_type'] == 'custom':
            continue
        set_id = set_info['aliases'][0]
        set_title = set_info.get('description', "")
        exps = set_info['replicate_exps']
        exp_details_recorded = False
        
        for exp_rep in exps:
            exp = exp_rep['replicate_exp']
            bio = exp_rep['bio_rep_no']
            tec = exp_rep['tec_rep_no'] 
            # record experiment set wide details from the first experiment
            if not exp_details_recorded:
                exp_info = ff_utils.get_metadata(exp,connection=ff, frame='embedded')
                cell_line = exp_info['biosample']['biosource'][0]['cell_line']
                mod = exp_info['biosample']['modifications_summary_short']
                treat = exp_info['biosample']["treatments_summary"]
                enzyme = exp_info['digestion_enzyme']['name']
                exp_details_recorded = True
                
            # get files
            exp_resp = ff_utils.get_metadata(exp,connection=ff)
            exp_files = exp_resp['files']
            
            for fastq_file in exp_files:
                file_resp = ff_utils.get_metadata(fastq_file, connection=ff)  
                #Some checks before running
                #check if status is deleted
                if file_resp['status'] == 'deleted':
                    print 'deleted file in experiment', file_resp['accession']
                    continue
                # skip pair no 2
                if file_resp.get('paired_end')=='2':
                    continue 
                sequencer = file_resp.get('instrument',"")
                file_alias = file_resp['aliases'][0]
                file_alias = file_alias[file_alias.find('SRR'):]
                file_alias = file_alias[:file_alias.find('_')]
                

                print '\t'.join([file_alias,pub_name,set_id,set_title,cell_line,enzyme,mod,treat,str(bio),str(tec),sequencer])


SRR2671226	Sanborn_et_al	dciclab:sanborn_rep01_h	in situ Hi-C on HAP-1 cells used to generate-wide wild-type Hi-C map	HAP-1	MboI	None	None	3	1	
SRR2671485	Sanborn_et_al	dciclab:sanborn_rep01	Capture Hi-C on wild-type HAP-1 cells	HAP-1	MboI	None	None	1	1	
SRR2671486	Sanborn_et_al	dciclab:sanborn_rep01	Capture Hi-C on wild-type HAP-1 cells	HAP-1	MboI	None	None	1	1	
SRR2671487	Sanborn_et_al	dciclab:sanborn_rep01	Capture Hi-C on wild-type HAP-1 cells	HAP-1	MboI	None	None	1	1	
SRR2671488	Sanborn_et_al	dciclab:sanborn_rep01	Capture Hi-C on wild-type HAP-1 cells	HAP-1	MboI	None	None	1	1	
